**Author:** Elias Rafael Ruiz-Morales

**Institution:** Wellcome Sanger institute

**July, 2023**

---

## Generate cellphoneDB input files (1) _meta.tsv and (2) _degs.tsv

In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata
import os
import sys
sys.executable

'/opt/conda/bin/python'

In [2]:
def recoverDEGs(DEGdf, infection, celltype, direction):

    
    ### Up reg
    ###
    if direction=='up':

        #selecting only up regulated genes
        tempUP = DEGdf[DEGdf['logFC'] > 0].copy()
        
        #making new columns
        tempUP['cell_type'] = celltype
        tempUP['cluster'] = celltype + '.' + infection
        tempUP['gene']=list(tempUP.index.values)

        # select significant less restrictive
        tempUP =tempUP[ tempUP.p_val_adj < 0.05]
        tempUP = tempUP[ tempUP.logFC > 0.1]

        
        return(tempUP)


    ### Down reg
    ###
    if direction=='down':
        #selecting only down regulated genes
        tempUP = DEGdf[DEGdf['logFC'] < 0].copy()
        
        #making new columns
        tempUP['cell_type'] = celltype
        tempUP['cluster'] = celltype + '.' + infection
        tempUP['gene']=list(tempUP.index.values)


        # select significant less restrictive
        tempUP =tempUP[ tempUP.p_val_adj < 0.05]
        tempUP = tempUP[ tempUP.logFC < -0.1]
        
        return(tempDwn)


## Prepare INPUT

### (1) Load annadata_ single-cell explants from malaria infection

In [5]:
adata = sc.read('../../data_integration/results/scVI/malaria_singleCell_24h.h5ad')

In [6]:
adata.obs['cell_type'].values.describe()

,counts,freqs
categories,,
EVT_1,301,0.006171
EVT_2,701,0.014372
Endo_f,5788,0.118667
F,9059,0.185730
F_p,566,0.011604
F_sm,483,0.009903
HBC,9000,0.184521
HBC_p,11,0.000226
PAMM1,2217,0.045454


In [7]:
adata.obs['stage_perInfection'].values.describe()

,counts,freqs
categories,,
Pf_24h,27100,0.555613
UI_Pf_24h,21675,0.444387


In [8]:
adata

AnnData object with n_obs × n_vars = 48775 × 36601
    obs: 'sample', 'stage', 'hpi', 'infection', 'percent_mito', 'n_counts', 'sample_barcode', 'assignment_SoC', 'donor_id', 'scrublet_score', 'scrublet_cluster_score', 'zscore', 'bh_pval', 'bonf_pval', 'S_score', 'G2M_score', 'phase', 'n_genes_by_counts', 'total_counts', 'total_counts_hs', 'pct_counts_hs', 'total_counts_tg', 'pct_counts_tg', 'Tg_infected', 'n_genes', '_scvi_batch', '_scvi_labels', '_scvi_local_l_mean', '_scvi_local_l_var', 'leiden_scvi', 'celltype_predictions', 'probabilities', 'scrublet_doublet', 'cell_type_2022', 'cell_type', 'souporcell_MFgenotype', 'MFgenotype', 'cell_type_broad', 'umap_density_Tg_infected', 'stage_perInfection', 'celltype-Stage', 'Tg_intracellular', 'celltype-Intracellular', 'Dev_Stage'
    var: 'gene_ids', 'feature_types', 'mean-0', 'std-0', 'mean-1', 'std-1', 'mean-2', 'std-2', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'Dev_Stage_colors', 'MFgenot

### DO NOT Subset dataset, use full matrix counts from infected manifod!! 

In [24]:
#Take only the INFECTED counts matrix
cellphoneDB_adata = adata[adata.obs['stage_perInfection']=='Pf_24h']

/opt/conda/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


In [25]:
#Take only the UNINFECTED counts matrix
#cellphoneDB_adataUI = adata[adata.obs['stage_perInfection']=='UI_Pf_24h']

In [26]:
cellphoneDB_adata.obs['cell_type']

Pla_HDBR11345034_AAACCCAAGGAGGTTC    EVT_1
Pla_HDBR11345034_AAACCCAAGGCCCAAA      HBC
Pla_HDBR11345034_AAACCCAAGTGGTCAG       PV
Pla_HDBR11345034_AAACCCACATACACCA        F
Pla_HDBR11345034_AAACCCATCAAATGAG      HBC
                                     ...  
Pla_HDBR13661570_TTTGTTGGTCTTGCGG     F_sm
Pla_HDBR13661570_TTTGTTGGTTCGGCGT      HBC
Pla_HDBR13661570_TTTGTTGGTTCTGAGT        F
Pla_HDBR13661570_TTTGTTGTCAGGAGAC       PV
Pla_HDBR13661570_TTTGTTGTCGCACGGT      HBC
Name: cell_type, Length: 27100, dtype: category
Categories (15, object): ['EVT_1', 'EVT_2', 'Endo_f', 'F', ..., 'VCT_CCC', 'VCT_fusing', 'VCT_p', 'iEVT']

In [27]:
cellphoneDB_adata.obs['donor_id'].values.describe()

,counts,freqs
categories,,
Hrv124,8212,0.303026
Hrv168,7112,0.262435
Hrv232,2376,0.087675
Hrv236,9400,0.346863


In [28]:
cellphoneDB_adata.obs['stage_perInfection'].values.describe()

,counts,freqs
categories,,
Pf_24h,27100,1.0


### Load *new* cell clusters annotation

In [29]:
### Add cluster subname such that this new name will match with DEG cluster name

#INFECTED
cellphoneDB_adata.obs['cell_type']= cellphoneDB_adata.obs.cell_type.astype('string')+'.'+cellphoneDB_adata.obs.infection.astype('string')


#UI
#cellphoneDB_adataUI.obs['cell_type']= cellphoneDB_adataUI.obs.cell_type.astype('string')+'.'+cellphoneDB_adataUI.obs.stage_perInfection.astype('string')

Trying to set attribute `.obs` of view, copying.


In [30]:
cellphoneDB_adata.obs['cell_type']

Pla_HDBR11345034_AAACCCAAGGAGGTTC    EVT_1.Pf
Pla_HDBR11345034_AAACCCAAGGCCCAAA      HBC.Pf
Pla_HDBR11345034_AAACCCAAGTGGTCAG       PV.Pf
Pla_HDBR11345034_AAACCCACATACACCA        F.Pf
Pla_HDBR11345034_AAACCCATCAAATGAG      HBC.Pf
                                       ...   
Pla_HDBR13661570_TTTGTTGGTCTTGCGG     F_sm.Pf
Pla_HDBR13661570_TTTGTTGGTTCGGCGT      HBC.Pf
Pla_HDBR13661570_TTTGTTGGTTCTGAGT        F.Pf
Pla_HDBR13661570_TTTGTTGTCAGGAGAC       PV.Pf
Pla_HDBR13661570_TTTGTTGTCGCACGGT      HBC.Pf
Name: cell_type, Length: 27100, dtype: string

In [31]:
cellphoneDB_adata

AnnData object with n_obs × n_vars = 27100 × 36601
    obs: 'sample', 'stage', 'hpi', 'infection', 'percent_mito', 'n_counts', 'sample_barcode', 'assignment_SoC', 'donor_id', 'scrublet_score', 'scrublet_cluster_score', 'zscore', 'bh_pval', 'bonf_pval', 'S_score', 'G2M_score', 'phase', 'n_genes_by_counts', 'total_counts', 'total_counts_hs', 'pct_counts_hs', 'total_counts_tg', 'pct_counts_tg', 'Tg_infected', 'n_genes', '_scvi_batch', '_scvi_labels', '_scvi_local_l_mean', '_scvi_local_l_var', 'leiden_scvi', 'celltype_predictions', 'probabilities', 'scrublet_doublet', 'cell_type_2022', 'cell_type', 'souporcell_MFgenotype', 'MFgenotype', 'cell_type_broad', 'umap_density_Tg_infected', 'stage_perInfection', 'celltype-Stage', 'Tg_intracellular', 'celltype-Intracellular', 'Dev_Stage'
    var: 'gene_ids', 'feature_types', 'mean-0', 'std-0', 'mean-1', 'std-1', 'mean-2', 'std-2', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'Dev_Stage_colors', 'MFgenot

In [32]:
sc.pp.normalize_per_cell(cellphoneDB_adata)
sc.pp.log1p(cellphoneDB_adata)                                                                                                            
# sc.pp.scale(adata, max_value=10)

/opt/conda/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


### The anndata object contains counts that have been normalized (per cell) and log-transformed, therefore save this as below

In [33]:
# counts input files 
cellphoneDB_adata.write('../data/malaria_inf_normlogTransformed_24h.h5ad')

... storing 'cell_type' as categorical


In [34]:
# meta file
df_meta = pd.DataFrame(data={'Cell':list(cellphoneDB_adata.obs.index),
                             'cell_type':[ str(i) for i in cellphoneDB_adata.obs['cell_type']] })
df_meta.set_index('Cell', inplace=True)
df_meta.to_csv('../data/malaria_inf_meta_24h.tsv', sep = '\t')

In [35]:
df_meta

,cell_type
Cell,
Pla_HDBR11345034_AAACCCAAGGAGGTTC,EVT_1.Pf
Pla_HDBR11345034_AAACCCAAGGCCCAAA,HBC.Pf
Pla_HDBR11345034_AAACCCAAGTGGTCAG,PV.Pf
Pla_HDBR11345034_AAACCCACATACACCA,F.Pf
Pla_HDBR11345034_AAACCCATCAAATGAG,HBC.Pf
...,...
Pla_HDBR13661570_TTTGTTGGTCTTGCGG,F_sm.Pf
Pla_HDBR13661570_TTTGTTGGTTCGGCGT,HBC.Pf
Pla_HDBR13661570_TTTGTTGGTTCTGAGT,F.Pf


In [36]:
cellphoneDB_adata.obs['cell_type']

Pla_HDBR11345034_AAACCCAAGGAGGTTC    EVT_1.Pf
Pla_HDBR11345034_AAACCCAAGGCCCAAA      HBC.Pf
Pla_HDBR11345034_AAACCCAAGTGGTCAG       PV.Pf
Pla_HDBR11345034_AAACCCACATACACCA        F.Pf
Pla_HDBR11345034_AAACCCATCAAATGAG      HBC.Pf
                                       ...   
Pla_HDBR13661570_TTTGTTGGTCTTGCGG     F_sm.Pf
Pla_HDBR13661570_TTTGTTGGTTCGGCGT      HBC.Pf
Pla_HDBR13661570_TTTGTTGGTTCTGAGT        F.Pf
Pla_HDBR13661570_TTTGTTGTCAGGAGAC       PV.Pf
Pla_HDBR13661570_TTTGTTGTCGCACGGT      HBC.Pf
Name: cell_type, Length: 27100, dtype: category
Categories (15, object): ['EVT_1.Pf', 'EVT_2.Pf', 'Endo_f.Pf', 'F.Pf', ..., 'VCT_CCC.Pf', 'VCT_fusing.Pf', 'VCT_p.Pf', 'iEVT.Pf']

### (2) Do DEGs individually from precalculated DEGs files

In [37]:
# PATH to immune cells
path='../../diffGeneExpression/macrophages/results/SC_limma_Immune_MAL_24h/'

### HB

In [38]:
HBDEG = pd.read_csv(path+'1_DEGs_SC_MAL_24h_HBC.tsv', header=0, index_col=0, sep='\t')

In [39]:
HBDEG_up= recoverDEGs(HBDEG, infection='Pf', celltype='HBC', direction='up')
HBDEG_up.head(5)

,logFC,logCPM,F,PValue,bonferroni,p_val_adj,cell_type,cluster,gene
ZNF341-AS1,0.940907,3.927395,199.622904,2.783162e-13,2.283585e-09,2.232571e-09,HBC,HBC.Pf,ZNF341-AS1
Z82244.2,2.443107,3.379129,178.314946,9.526965e-13,7.816875e-09,2.605625e-09,HBC,HBC.Pf,Z82244.2
PTGIR,1.725812,3.955854,168.512944,1.754124e-12,1.439258e-08,3.598146e-09,HBC,HBC.Pf,PTGIR
MCRIP2,0.964274,5.891893,165.032206,2.195188e-12,1.801151e-08,3.602303e-09,HBC,HBC.Pf,MCRIP2
COL27A1,1.343494,3.982235,161.456229,2.776139e-12,2.277822e-08,3.796370e-09,HBC,HBC.Pf,COL27A1


In [40]:
# make sure DEGs only includes samples in meta
cl2include = set(cellphoneDB_adata.obs.cell_type.tolist())
idx = np.array([i in cl2include for i in HBDEG_up.cluster])
HBDEG_up = HBDEG_up[ idx ]

In [41]:
##  For cellphoneDB DEG mode, the 1st column = cluster; 2nd column = gene.
##  All other columns are ignored.
cncol = HBDEG_up.columns.tolist()
cncol = [item for item in cncol if item not in ['cluster', 'gene'] ]
HBDEG_up= HBDEG_up[ ['cluster', 'gene']+cncol ]
pd.DataFrame(HBDEG_up).to_csv('../data/malaria_inf_HBC_DEGsup.tsv', sep='\t', index=False)

In [42]:
#HBDEG_up=pd.read_table('../data/malaria_inf_HBC_DEGsup.tsv')
#HBDEG_up.loc[['VEGFA']]
HBDEG_up.loc[HBDEG_up['gene'] == 'VEGFA']

,cluster,gene,logFC,logCPM,F,PValue,bonferroni,p_val_adj,cell_type
VEGFA,HBC.Pf,VEGFA,0.614822,6.749733,35.556577,0.000003,0.028104,0.00008,HBC


### PAMM1

In [43]:
PAMM1DEG = pd.read_csv(path+'2_DEGs_SC_MAL_24h_PAMM1.tsv', header=0, index_col=0, sep='\t')

In [44]:
PAMM1DEG_up= recoverDEGs(PAMM1DEG, infection='Pf', celltype='PAMM1', direction='up')
PAMM1DEG_up.head(5)

,logFC,logCPM,F,PValue,bonferroni,p_val_adj,cell_type,cluster,gene
C4orf3,0.616426,7.769232,63.068401,2.753980e-08,0.000159,0.000068,PAMM1,PAMM1.Pf,C4orf3
FAM162A,1.298828,6.355774,63.024569,2.771374e-08,0.000160,0.000068,PAMM1,PAMM1.Pf,FAM162A
MYO1B,1.025988,5.912743,63.319447,3.561647e-08,0.000205,0.000068,PAMM1,PAMM1.Pf,MYO1B
TNF,0.910981,6.178153,60.599354,5.230316e-08,0.000301,0.000075,PAMM1,PAMM1.Pf,TNF
HILPDA,1.421535,7.880796,48.940179,3.177228e-07,0.001829,0.000305,PAMM1,PAMM1.Pf,HILPDA


In [45]:
# make sure DEGs only includes samples in meta
cl2include = set(cellphoneDB_adata.obs.cell_type.tolist())
idx = np.array([i in cl2include for i in PAMM1DEG_up.cluster])
PAMM1DEG_up = PAMM1DEG_up[ idx ]

In [46]:
# 1st column = cluster; 2nd column = gene 
cncol = PAMM1DEG_up.columns.tolist()
cncol = [item for item in cncol if item not in ['cluster', 'gene'] ]
PAMM1DEG_up= PAMM1DEG_up[ ['cluster', 'gene']+cncol ]
pd.DataFrame(PAMM1DEG_up).to_csv('../data/malaria_inf_PAMM1_DEGsup.tsv', sep='\t', index=False)

In [47]:
PAMM1DEG_up.head(5)

,cluster,gene,logFC,logCPM,F,PValue,bonferroni,p_val_adj,cell_type
C4orf3,PAMM1.Pf,C4orf3,0.616426,7.769232,63.068401,2.753980e-08,0.000159,0.000068,PAMM1
FAM162A,PAMM1.Pf,FAM162A,1.298828,6.355774,63.024569,2.771374e-08,0.000160,0.000068,PAMM1
MYO1B,PAMM1.Pf,MYO1B,1.025988,5.912743,63.319447,3.561647e-08,0.000205,0.000068,PAMM1
TNF,PAMM1.Pf,TNF,0.910981,6.178153,60.599354,5.230316e-08,0.000301,0.000075,PAMM1
HILPDA,PAMM1.Pf,HILPDA,1.421535,7.880796,48.940179,3.177228e-07,0.001829,0.000305,PAMM1


In [106]:
# PATH to FibroEndo cells DGE tables
path='../../diffGeneExpression/fibroEndo/results/SC_limma_fibroEndo_MAL_24h/'

### F

In [49]:
FDEG = pd.read_csv(path+'/1_DEGs_SC_MAL_24h_F.tsv', header=0, index_col=0, sep='\t')

In [50]:
FDEG_up= recoverDEGs(FDEG, infection='Pf', celltype='F', direction='up')
FDEG_up.head(5)

,logFC,logCPM,F,PValue,bonferroni,p_val_adj,cell_type,cluster,gene
HLA-DQB1,1.740492,4.800328,268.175627,6.820775e-13,4.961432e-09,4.961432e-09,F,F.Pf,HLA-DQB1
IL1A,1.410089,6.560718,169.960655,1.531624e-12,1.114104e-08,5.570518e-09,F,F.Pf,IL1A
SLC43A3,0.685064,5.872149,159.081183,3.119786e-12,2.269333e-08,7.162107e-09,F,F.Pf,SLC43A3
C15orf48,0.995682,7.925145,155.652466,3.938470e-12,2.864843e-08,7.162107e-09,F,F.Pf,C15orf48
SLC2A1,1.027442,6.796381,151.772000,5.155001e-12,3.749748e-08,7.499496e-09,F,F.Pf,SLC2A1


In [51]:
# make sure DEGs only includes samples in meta
cl2include = set(cellphoneDB_adata.obs.cell_type.tolist())
idx = np.array([i in cl2include for i in FDEG_up.cluster])
FDEG_up = FDEG_up[ idx ]

In [52]:
# 1st column = cluster; 2nd column = gene 
cncol = FDEG_up.columns.tolist()
cncol = [item for item in cncol if item not in ['cluster', 'gene'] ]
FDEG_up= FDEG_up[ ['cluster', 'gene']+cncol ]
pd.DataFrame(FDEG_up).to_csv('../data/malaria_inf_F_DEGsup.tsv', sep='\t', index=False)

In [53]:
FDEG_up.head()

,cluster,gene,logFC,logCPM,F,PValue,bonferroni,p_val_adj,cell_type
HLA-DQB1,F.Pf,HLA-DQB1,1.740492,4.800328,268.175627,6.820775e-13,4.961432e-09,4.961432e-09,F
IL1A,F.Pf,IL1A,1.410089,6.560718,169.960655,1.531624e-12,1.114104e-08,5.570518e-09,F
SLC43A3,F.Pf,SLC43A3,0.685064,5.872149,159.081183,3.119786e-12,2.269333e-08,7.162107e-09,F
C15orf48,F.Pf,C15orf48,0.995682,7.925145,155.652466,3.938470e-12,2.864843e-08,7.162107e-09,F
SLC2A1,F.Pf,SLC2A1,1.027442,6.796381,151.772000,5.155001e-12,3.749748e-08,7.499496e-09,F


In [54]:
FDEG_up.loc[['CD99']]

,cluster,gene,logFC,logCPM,F,PValue,bonferroni,p_val_adj,cell_type
CD99,F.Pf,CD99,0.204357,7.025717,6.955509,0.01427,1.0,0.043196,F


### PV

In [55]:
PVDEG = pd.read_csv(path+'/2_DEGs_SC_MAL_24h_PV.tsv', header=0, index_col=0, sep='\t')

In [56]:
PVDEG_up= recoverDEGs(PVDEG, infection='Pf', celltype='PV', direction='up')
PVDEG_up.head(5)

,logFC,logCPM,F,PValue,bonferroni,p_val_adj,cell_type,cluster,gene
C15orf48,1.387066,6.225848,172.385507,6.192700e-13,5.426044e-09,5.426044e-09,PV,PV.Pf,C15orf48
SAMSN1,1.233680,4.497978,119.561670,3.474524e-11,3.044378e-07,9.219040e-08,PV,PV.Pf,SAMSN1
SNX10,1.564362,3.189589,124.008690,2.347921e-11,2.057248e-07,9.219040e-08,PV,PV.Pf,SNX10
ICAM1,1.099168,7.059811,117.435461,4.208646e-11,3.687616e-07,9.219040e-08,PV,PV.Pf,ICAM1
ALOX5AP,1.159447,3.605170,146.496630,6.942756e-11,6.083243e-07,1.216649e-07,PV,PV.Pf,ALOX5AP


In [57]:
# make sure DEGs only includes samples in meta
cl2include = set(cellphoneDB_adata.obs.cell_type.tolist())
idx = np.array([i in cl2include for i in PVDEG_up.cluster])
PVDEG_up = PVDEG_up[ idx ]

In [58]:
# 1st column = cluster; 2nd column = gene 
cncol = PVDEG_up.columns.tolist()
cncol = [item for item in cncol if item not in ['cluster', 'gene'] ]
PVDEG_up= PVDEG_up[ ['cluster', 'gene']+cncol ]
pd.DataFrame(PVDEG_up).to_csv('../data/malaria_inf_PV_DEGsup.tsv', sep='\t', index=False)

In [59]:
PVDEG_up.head()

,cluster,gene,logFC,logCPM,F,PValue,bonferroni,p_val_adj,cell_type
C15orf48,PV.Pf,C15orf48,1.387066,6.225848,172.385507,6.192700e-13,5.426044e-09,5.426044e-09,PV
SAMSN1,PV.Pf,SAMSN1,1.233680,4.497978,119.561670,3.474524e-11,3.044378e-07,9.219040e-08,PV
SNX10,PV.Pf,SNX10,1.564362,3.189589,124.008690,2.347921e-11,2.057248e-07,9.219040e-08,PV
ICAM1,PV.Pf,ICAM1,1.099168,7.059811,117.435461,4.208646e-11,3.687616e-07,9.219040e-08,PV
ALOX5AP,PV.Pf,ALOX5AP,1.159447,3.605170,146.496630,6.942756e-11,6.083243e-07,1.216649e-07,PV


### Endo_f

In [60]:
EndoDEG = pd.read_csv(path+'/3_DEGs_SC_MAL_24h_Endof.tsv', header=0, index_col=0, sep='\t')

In [61]:
EndoDEG_up= recoverDEGs(EndoDEG, infection='Pf', celltype='Endo_f', direction='up')
EndoDEG_up.head(5)

,logFC,logCPM,F,PValue,bonferroni,p_val_adj,cell_type,cluster,gene
IL1RN,2.190200,3.914339,184.166411,2.181283e-13,1.816572e-09,1.816572e-09,Endo_f,Endo_f.Pf,IL1RN
CCL3L1,2.244097,4.111346,146.678153,2.917805e-12,2.429948e-08,1.214974e-08,Endo_f,Endo_f.Pf,CCL3L1
IL1A,1.178834,5.366196,123.323318,1.990271e-11,1.657498e-07,5.057966e-08,Endo_f,Endo_f.Pf,IL1A
FTH1,0.839001,12.135905,121.088272,2.429378e-11,2.023186e-07,5.057966e-08,Endo_f,Endo_f.Pf,FTH1
CCL5,2.153293,4.829508,111.313957,6.031100e-11,5.022700e-07,1.004540e-07,Endo_f,Endo_f.Pf,CCL5


In [62]:
# make sure DEGs only includes samples in meta
cl2include = set(cellphoneDB_adata.obs.cell_type.tolist())
idx = np.array([i in cl2include for i in EndoDEG_up.cluster])
EndoDEG_up = EndoDEG_up[ idx ]

In [63]:
# 1st column = cluster; 2nd column = gene 
cncol = EndoDEG_up.columns.tolist()
cncol = [item for item in cncol if item not in ['cluster', 'gene'] ]
EndoDEG_up= EndoDEG_up[ ['cluster', 'gene']+cncol ]
pd.DataFrame(EndoDEG_up).to_csv('../data/malaria_inf_Endof_DEGsup.tsv', sep='\t', index=False)

In [64]:
EndoDEG_up.head()

,cluster,gene,logFC,logCPM,F,PValue,bonferroni,p_val_adj,cell_type
IL1RN,Endo_f.Pf,IL1RN,2.190200,3.914339,184.166411,2.181283e-13,1.816572e-09,1.816572e-09,Endo_f
CCL3L1,Endo_f.Pf,CCL3L1,2.244097,4.111346,146.678153,2.917805e-12,2.429948e-08,1.214974e-08,Endo_f
IL1A,Endo_f.Pf,IL1A,1.178834,5.366196,123.323318,1.990271e-11,1.657498e-07,5.057966e-08,Endo_f
FTH1,Endo_f.Pf,FTH1,0.839001,12.135905,121.088272,2.429378e-11,2.023186e-07,5.057966e-08,Endo_f
CCL5,Endo_f.Pf,CCL5,2.153293,4.829508,111.313957,6.031100e-11,5.022700e-07,1.004540e-07,Endo_f


In [92]:
# PATH to Trophoblast cells DGE tables
path='../../diffGeneExpression/trophoblast/results/SC_limma_Trophoblast_MAL_24h'

### VCT_fusing

In [93]:
VCTfDEG = pd.read_csv(path+'/1_DEGs_SC_MAL_24h_VCT_fusing.tsv', header=0, index_col=0, sep='\t')

In [94]:
VCTfDEG_up= recoverDEGs(VCTfDEG, infection='Pf', celltype='VCT_fusing', direction='up')
VCTfDEG_up.head(5)

,logFC,logCPM,F,PValue,bonferroni,p_val_adj,cell_type,cluster,gene
MMP24OS,0.374704,5.726263,90.631744,1.225836e-09,0.000010,0.000010,VCT_fusing,VCT_fusing.Pf,MMP24OS
GUK1,0.230347,7.289740,72.022147,1.058798e-08,0.000084,0.000017,VCT_fusing,VCT_fusing.Pf,GUK1
ERRFI1,0.519455,8.211099,58.491610,6.755355e-08,0.000535,0.000049,VCT_fusing,VCT_fusing.Pf,ERRFI1
NFKBIZ,0.328492,5.681678,58.968408,6.296046e-08,0.000499,0.000049,VCT_fusing,VCT_fusing.Pf,NFKBIZ
ATOX1,0.257473,6.586556,57.898700,7.377820e-08,0.000584,0.000049,VCT_fusing,VCT_fusing.Pf,ATOX1


In [95]:
# make sure DEGs only includes samples in meta
cl2include = set(cellphoneDB_adata.obs.cell_type.tolist())
idx = np.array([i in cl2include for i in VCTfDEG_up.cluster])
VCTfVCTfDEG_upDEG = VCTfDEG_up[ idx ]

In [96]:
# 1st column = cluster; 2nd column = gene 
cncol = VCTfDEG_up.columns.tolist()
cncol = [item for item in cncol if item not in ['cluster', 'gene'] ]
VCTfDEG_up= VCTfDEG_up[ ['cluster', 'gene']+cncol ]
pd.DataFrame(VCTfDEG_up).to_csv('../data/malaria_inf_VCTfusing_DEGsup.tsv', sep='\t', index=False)

In [97]:
VCTfDEG_up.head()

,cluster,gene,logFC,logCPM,F,PValue,bonferroni,p_val_adj,cell_type
MMP24OS,VCT_fusing.Pf,MMP24OS,0.374704,5.726263,90.631744,1.225836e-09,0.000010,0.000010,VCT_fusing
GUK1,VCT_fusing.Pf,GUK1,0.230347,7.289740,72.022147,1.058798e-08,0.000084,0.000017,VCT_fusing
ERRFI1,VCT_fusing.Pf,ERRFI1,0.519455,8.211099,58.491610,6.755355e-08,0.000535,0.000049,VCT_fusing
NFKBIZ,VCT_fusing.Pf,NFKBIZ,0.328492,5.681678,58.968408,6.296046e-08,0.000499,0.000049,VCT_fusing
ATOX1,VCT_fusing.Pf,ATOX1,0.257473,6.586556,57.898700,7.377820e-08,0.000584,0.000049,VCT_fusing


In [98]:
VCTfDEG_up.loc[['SPP1']]

,cluster,gene,logFC,logCPM,F,PValue,bonferroni,p_val_adj,cell_type
SPP1,VCT_fusing.Pf,SPP1,1.152154,6.005685,18.427151,0.000249,1.0,0.004494,VCT_fusing


### VCT

In [99]:
VCTDEG = pd.read_csv(path+'/2_DEGs_SC_MAL_24h_VCT.tsv', header=0, index_col=0, sep='\t')

In [100]:
VCTDEG_up= recoverDEGs(VCTDEG, infection='Pf', celltype='VCT', direction='up')
VCTDEG_up.head(5)

,logFC,logCPM,F,PValue,bonferroni,p_val_adj,cell_type,cluster,gene
CCL3,1.994071,5.772547,155.362130,9.110064e-13,9.107330e-09,9.107330e-09,VCT,VCT.Pf,CCL3
SPP1,1.706711,5.689939,67.544276,7.463222e-09,7.460983e-05,2.486994e-05,VCT,VCT.Pf,SPP1
HMGN2,0.439966,9.238759,60.852914,2.051519e-08,2.050904e-04,3.257878e-05,VCT,VCT.Pf,HMGN2
TMEM265,0.727250,5.965756,60.179202,2.281199e-08,2.280514e-04,3.257878e-05,VCT,VCT.Pf,TMEM265
CCL20,1.452804,5.077363,54.852381,5.443452e-08,5.441819e-04,6.802274e-05,VCT,VCT.Pf,CCL20


In [101]:
# make sure DEGs only includes samples in meta
cl2include = set(cellphoneDB_adata.obs.cell_type.tolist())
idx = np.array([i in cl2include for i in VCTDEG_up.cluster])
VCTDEG_up = VCTDEG_up[ idx ]

In [102]:
# 1st column = cluster; 2nd column = gene 
cncol = VCTDEG_up.columns.tolist()
cncol = [item for item in cncol if item not in ['cluster', 'gene'] ]
VCTDEG_up= VCTDEG_up[ ['cluster', 'gene']+cncol ]
pd.DataFrame(VCTDEG_up).to_csv('../data/malaria_inf_VCT_DEGsup.tsv', sep='\t', index=False)

In [103]:
VCTDEG_up.head()

,cluster,gene,logFC,logCPM,F,PValue,bonferroni,p_val_adj,cell_type
CCL3,VCT.Pf,CCL3,1.994071,5.772547,155.362130,9.110064e-13,9.107330e-09,9.107330e-09,VCT
SPP1,VCT.Pf,SPP1,1.706711,5.689939,67.544276,7.463222e-09,7.460983e-05,2.486994e-05,VCT
HMGN2,VCT.Pf,HMGN2,0.439966,9.238759,60.852914,2.051519e-08,2.050904e-04,3.257878e-05,VCT
TMEM265,VCT.Pf,TMEM265,0.727250,5.965756,60.179202,2.281199e-08,2.280514e-04,3.257878e-05,VCT
CCL20,VCT.Pf,CCL20,1.452804,5.077363,54.852381,5.443452e-08,5.441819e-04,6.802274e-05,VCT


In [104]:
VCTDEG_up.loc[['SPP1']]

,cluster,gene,logFC,logCPM,F,PValue,bonferroni,p_val_adj,cell_type
SPP1,VCT.Pf,SPP1,1.706711,5.689939,67.544276,7.463222e-09,0.000075,0.000025,VCT


### (3) Run cellphoneDB